In [17]:
# https://google.github.io/adk-docs/sessions/state/#organizing-state-with-prefixes-scope-matters

state1 = 'test'

formatting_instructions = f"""
three {{{state1}}} # take it from the .state; it is mandatory
three? {{{state1}?}} # take it from the .state; it is optional
two {{state1}} # consider it as exact value in brackets
one {state1} ## consider it as a simple variable defined somewhere else
"""
print(formatting_instructions)


three {test} # take it from the .state; it is mandatory
three? {test?} # take it from the .state; it is optional
two {state1} # consider it as exact value in brackets
one test ## consider it as a simple variable defined somewhere else

{test?}



In [20]:
"{{{}?}}".format(state1)

'{test?}'

In [21]:
# NEXT STEPS:
## V1 ##
# - test if works (when wrong movie is asked for)
# - create a workflow diagram + vid and short desc
# - focus on the srcaping agent which doesn't know how to scrap vod STREAMING only -> consider be more specific and to focus on a particula <div> only!
#              try the classic fetching with request and bs4 - read below
    -->> there is an issue with the first agent which does't pass the movie title to the next agent for some reason... maybe force the sequential agent or sth...'
## V2 ##
# - deploy
# - add more movies at the same time/ parallel
## V3 ##
# - adding before_agent_callback = increment_attempt instead of the tool usage

In [1]:

from bs4 import BeautifulSoup
import requests
import re
from firecrawl import FirecrawlApp
from typing import List

def scrape_website(url: str) -> List[str]:
    """
    Extract VOD streaming providers from a JustWatch movie page.

    Args:
        url: JustWatch movie URL (e.g., https://www.justwatch.com/us/movie/dune-part-two-2023)

    Returns:
        List of streaming provider names (e.g., ['HBO Max', 'Netflix'])

    Raises:
        requests.RequestException: If the HTTP request fails
        ValueError: If the URL is invalid or no streaming section found
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        raise requests.RequestException(f"Failed to fetch URL: {e}")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the STREAM buybox row
    stream_div = soup.find('div', class_='buybox-row stream')

    if not stream_div:
        return []

    # Extract all provider titles from provider-icon divs
    providers = [
        icon.get('title')
        for icon in stream_div.find_all('div', class_='provider-icon')
        if icon.get('title')
    ]

    return providers


In [2]:
scrape_website('https://www.justwatch.com/us/movie/dune-part-two-2023')

[]

In [5]:
url = 'https://www.justwatch.com/us/movie/dune-part-two-2023'

from bs4 import BeautifulSoup
import requests

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
except requests.RequestException as e:
    raise requests.RequestException(f"Failed to fetch URL: {e}")

soup = BeautifulSoup(response.content, 'html.parser')


In [6]:
soup

<!DOCTYPE html>
<html data-vue-meta="%7B%22dir%22:%7B%22ssr%22:%22ltr%22%7D,%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-vue-meta-server-rendered="" dir="ltr" lang="en"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8" data-vue-meta="ssr"/><meta content="IE=edge" data-vue-meta="ssr" http-equiv="X-UA-Compatible"/><meta content="viewport-fit=cover, width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" data-vue-meta="ssr" name="viewport"/><meta content="JustWatch" data-vue-meta="ssr" property="og:site_name"/><meta content="794243977319785" data-vue-meta="ssr" property="fb:app_id"/><meta content="yes" data-vue-meta="ssr" name="mobile-web-app-capable"/><meta content="yes" data-vue-meta="ssr" name="apple-mobile-web-app-capable"/><meta content="#182531" data-vue-meta="ssr" name="theme-color"/><meta content="AkO0-dINGfhSod2X6LeYIp9hwZc8ShczjzJA2a47dME" data-vue-meta="ssr" name="google-site-verificat

In [12]:
stream_div = soup.find('img', class_='provider-icon square icon')
print(stream_div)

None


In [20]:
from playwright.sync_api import sync_playwright, TimeoutError as PlaywrightTimeout
from bs4 import BeautifulSoup
from typing import List

def extract_streaming_providers(url: str) -> List[str]:
    """
    Extract VOD streaming providers from a JustWatch movie page using Playwright.

    Args:
        url: JustWatch movie URL (e.g., https://www.justwatch.com/us/movie/dune-part-two-2023)

    Returns:
        List of streaming provider names (e.g., ['HBO Max', 'Netflix'])

    Raises:
        PlaywrightTimeout: If the page fails to load or streaming section not found
        Exception: For other errors during scraping
    """
    with sync_playwright() as p:
        # Launch browser in headless mode
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        try:
            # Navigate to the URL
            page.goto(url, wait_until='networkidle', timeout=30000)

            # Wait for the streaming section to appear
            page.wait_for_selector('.buybox-row.stream', timeout=10000)

            # Get the rendered HTML
            html = page.content()
            soup = BeautifulSoup(html, 'html.parser')

            # Find the STREAM buybox row
            stream_div = soup.find('div', class_='buybox-row stream')

            if not stream_div:
                return []

            # Extract all provider titles from provider-icon divs
            providers = [
                icon.get('title')
                for icon in stream_div.find_all('div', class_='provider-icon')
                if icon.get('title')
            ]

            return providers

        except PlaywrightTimeout:
            # No streaming section found or page didn't load
            return []

        finally:
            browser.close()

extract_streaming_providers('https://www.justwatch.com/us/movie/dune-part-two-2023')

In [21]:
extract_streaming_providers('https://www.justwatch.com/us/movie/dune-part-two-2023')

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [23]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from typing import List

async def extract_streaming_providers(url: str) -> List[str]:
    """
    Extract VOD streaming providers from a JustWatch movie page using Playwright.

    Args:
        url: JustWatch movie URL (e.g., https://www.justwatch.com/us/movie/dune-part-two-2023)

    Returns:
        List of streaming provider names (e.g., ['HBO Max', 'Netflix'])
    """
    async with async_playwright() as p:
        # Launch browser in headless mode
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            # Navigate to the URL
            await page.goto(url, wait_until='networkidle', timeout=30000)

            # Wait for the streaming section to appear
            await page.wait_for_selector('.buybox-row.stream', timeout=10000)

            # Get the rendered HTML
            html = await page.content()
            soup = BeautifulSoup(html, 'html.parser')

            # Find the STREAM buybox row
            stream_div = soup.find('div', class_='buybox-row stream')

            if not stream_div:
                return []

            # Extract all provider titles from provider-icon divs
            providers = [
                icon.get('title')
                for icon in stream_div.find_all('div', class_='provider-icon')
                if icon.get('title')
            ]

            return providers

        except Exception:
            return []

        finally:
            await browser.close()

# Run it in Jupyter with await
await extract_streaming_providers('https://www.justwatch.com/us/movie/dune-part-two-2023')

NotImplementedError: 